In [2]:
#https://djajafer.medium.com/multi-class-text-classification-with-keras-and-lstm-4c5525bef592
import gzip
import json
import pandas as pd
from sklearn.model_selection import train_test_split
import csv
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Flatten, LSTM, Dropout, Activation, Embedding, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
import pickle
from datetime import datetime

# Import Data

In [11]:
data = []
with gzip.open('../data/reviews_Video_Games_5.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip()))
df = pd.DataFrame.from_dict(data).dropna().reset_index(drop=True)
X = df.loc[:,'reviewText']
y = df.loc[:,'overall']

# Clean Data

In [12]:
def fivestar_to_binary(number):
    """Convert 1-5 rating scale to binary 'good' or 'bad' reviews"""
    return 1 if number>3 else 0

In [13]:
y = y.apply(fivestar_to_binary)

In [36]:
bad_reviews = np.where(y == 0)[0][:50_000]
good_reviews = np.where(y == 1)[0][:50_000]
balanced_reviews = np.append(bad_reviews,good_reviews)

In [39]:
# only work with a subset of the data for the purpose of speed
X = [X[i] for i in balanced_reviews]
y = [y[i] for i in balanced_reviews]

In [42]:
# split into train, test, validation sets (60, 20, 20% respectively)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25)

Note that we will not use the test set for anything other than evaluation of our text attacks.

In [45]:
#save test data for later
with open('../data/traindata.pickle', 'wb') as wf:
    traindata = (X_train,y_train)
    pickle.dump(traindata, wf)
    
with open('../data/valdata.pickle', 'wb') as wf:
    valdata = (X_val,y_val)
    pickle.dump(testdata, wf)

with open('../data/testdata.pickle', 'wb') as wf:
    testdata = (X_test,y_test)
    pickle.dump(testdata, wf)